# LINEAR REGRESSION

## Data loading

#### Dataset
- Shape: 506 rows × 14 columns
- Target column: MEDV = median value of owner-occupied homes (in $1,000s)
#### Missing values
- Some columns had missing values (example: CRIM had only 486 non-null values out of 506).
- I handled missing values using mean imputation per numeric column because linear models don’t accept NaNs, and mean imputation is a very easy method.

In [1]:
import zipfile
import math
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [2]:
class CustomLinearRegressionGD:
    """
    custom linear regression trained with batch gradient descent.

    theory (from Sigmoid ML Handbook):
    we assume a linear model y_hat = Xw, and we want to find weights w
    that minimize the sum of squared errors:
        f(w) = (Xw - y)^T (Xw - y)
    the gradient of this cost wrt w is:
        grad = 2 * X^T (Xw - y)
    we iteratively update:
        w <- w - alpha * grad
    until convergence / fixed iterations.

    other tweaks:
    -feature standardization (mean 0, std 1) for stability.
    -explicit bias term (a column of 1s).
    -deterministic random_state for reproducible initialization.
    -score() method that returns R^2, so it matches sklearn's .score().
    """

    def __init__(self, learning_rate=0.01, n_iterations=5000, random_state=42):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.random_state = random_state

        #will be learned in fit()
        self.w_ = None      #weight vector INCLUDING bias term at the end
        self.mean_ = None   #per-feature mean
        self.std_ = None    #per-feature std (for scaling)

    def _prepare_features_fit(self, X):
        """
        standardize features using training data stats and
        append a bias column of 1s.
        """
        rng = np.random.default_rng(self.random_state)

        #compute and store scaling parameters
        self.mean_ = X.mean(axis=0)
        self.std_ = X.std(axis=0)
        #avoid division by zero if a column is constant
        self.std_[self.std_ == 0] = 1.0

        #standardize
        X_scaled = (X - self.mean_) / self.std_

        #add bias column (all 1s) as last column
        ones = np.ones((X_scaled.shape[0], 1))
        X_aug = np.concatenate([X_scaled, ones], axis=1)

        #initialize weights randomly (small values)
        self.w_ = rng.normal(loc=0.0, scale=0.01, size=(X_aug.shape[1], 1))

        return X_aug

    def _prepare_features_predict(self, X):
        """
        standardize features using the *stored* mean_ and std_,
        then append bias column of 1s (same trick as in training).
        """
        X_scaled = (X - self.mean_) / self.std_
        ones = np.ones((X_scaled.shape[0], 1))
        X_aug = np.concatenate([X_scaled, ones], axis=1)
        return X_aug

    def fit(self, X, y):
        """
        fit the model using full-batch gradient descent.

        parameters
        ----------
        X : np.ndarray, shape (n_samples, n_features)
            Training features.
        y : array-like, shape (n_samples,) or (n_samples, 1)
            Target values (MEDV in our case).
        """
        #prepare features (scaling + bias) and ensure y is column vector
        X_aug = self._prepare_features_fit(X)
        y = y.reshape(-1, 1)

        n_samples = X_aug.shape[0]

        #gradient Descent loop
        for _ in range(self.n_iterations):
            #forward pass: predictions
            y_pred = X_aug @ self.w_

            #gradient of MSE-like objective 
            gradient = (2.0 / n_samples) * (X_aug.T @ (y_pred - y))

            #update rule: w <- w - alpha * grad
            self.w_ -= self.learning_rate * gradient

        return self

    def predict(self, X):
        """
        predict using learned weights.
        """
        X_aug = self._prepare_features_predict(X)
        return X_aug @ self.w_

    def score(self, X, y):
        """
        compute R^2, same definition as sklearn's LinearRegression.score.
        R^2 = 1 - SS_res / SS_tot
        """
        y = y.reshape(-1, 1)
        y_pred = self.predict(X)

        ss_res = np.sum((y - y_pred) ** 2)
        ss_tot = np.sum((y - y.mean()) ** 2)
        r2 = 1.0 - ss_res / ss_tot
        return r2


def load_and_clean_data(zip_path):
    """
    load HousingData.csv from the given ZIP file, fill missing numeric values
    with column means, and return the cleaned DataFrame.
    """
    with zipfile.ZipFile(zip_path, 'r') as z:
        #find CSV inside zip
        csv_files = [f for f in z.namelist() if f.endswith(".csv")]
        if not csv_files:
            raise FileNotFoundError("no CSV file found in archive.")
        data_file = csv_files[0]

        with z.open(data_file) as f:
            df = pd.read_csv(f)

    #mean imputation for numeric columns
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        df[col] = df[col].fillna(df[col].mean())

    return df


def compute_correlations(df, target_col="MEDV"):
    """
    compute Pearson correlation with the target column,
    return a DataFrame sorted by absolute correlation descending.
    """
    corr_matrix = df.corr(numeric_only=True)
    #drop target itself
    target_corr = corr_matrix[target_col].drop(target_col)

    corr_df = pd.DataFrame({
        "feature": target_corr.index,
        "corr_with_MEDV": target_corr.values,
        "abs_corr": np.abs(target_corr.values)
    }).sort_values(by="abs_corr", ascending=False)

    return corr_df


def select_feature_subset(corr_df, low=0.5, high=0.8):
    """
    select feature names whose absolute correlation with the target is
    between 'low' and 'high', inclusive.
    """
    mask = (corr_df["abs_corr"] >= low) & (corr_df["abs_corr"] <= high)
    return corr_df.loc[mask, "feature"].tolist()


def train_and_evaluate_models(X_train, X_test, y_train, y_test):
    """
    train both sklearn's LinearRegression and our CustomLinearRegressionGD
    on the provided train/test split. return their R^2 scores.
    """
    #Sklearn OLS 
    skl_lr = LinearRegression()
    skl_lr.fit(X_train, y_train)
    r2_skl = skl_lr.score(X_test, y_test)

    #Custom GD Linear Regression
    custom_lr = CustomLinearRegressionGD(
        learning_rate=0.01,
        n_iterations=5000,
        random_state=42
    )
    custom_lr.fit(X_train, y_train)
    r2_custom = custom_lr.score(X_test, y_test)

    return r2_skl, r2_custom

In [3]:
def main():
    #load + clean data
    df = load_and_clean_data("data/archive.zip")

    #correlation analysis
    corr_df = compute_correlations(df, target_col="MEDV")
    print("Correlation with MEDV (sorted by |corr|):\n", corr_df, "\n")

    #prepare full and subset feature matrices
    target = "MEDV"
    X_full = df.drop(columns=[target]).values #all predictors
    y_full = df[target].values.reshape(-1, 1)

    subset_features = select_feature_subset(corr_df, low=0.5, high=0.8)
    print("Selected high-corr features (|corr| in [0.5, 0.8]):", subset_features)

    X_sub = df[subset_features].values #only top drivers
    y_sub = y_full

    #train/test split for BOTH datasets
    X_train_full, X_test_full, y_train_full, y_test_full = train_test_split(
        X_full, y_full, test_size=0.2, random_state=42
    )

    X_train_sub, X_test_sub, y_train_sub, y_test_sub = train_test_split(
        X_sub, y_sub, test_size=0.2, random_state=42
    )

    #train & evaluate on full feature set
    r2_full_skl, r2_full_custom = train_and_evaluate_models(
        X_train_full, X_test_full, y_train_full, y_test_full
    )

    #train & evaluate on subset feature set
    r2_sub_skl, r2_sub_custom = train_and_evaluate_models(
        X_train_sub, X_test_sub, y_train_sub, y_test_sub
    )

    #summarize results
    results_df = pd.DataFrame({
        "dataset": ["full", "full", "subset", "subset"],
        "model": ["sklearn_OLS", "custom_GD", "sklearn_OLS", "custom_GD"],
        "R2_test": [r2_full_skl, r2_full_custom, r2_sub_skl, r2_sub_custom]
    })

    print("\n=== Final R^2 comparison table ===")
    print(results_df.to_string(index=False))


if __name__ == "__main__":
    main()

Correlation with MEDV (sorted by |corr|):
     feature  corr_with_MEDV  abs_corr
12    LSTAT       -0.721975  0.721975
5        RM        0.695360  0.695360
10  PTRATIO       -0.507787  0.507787
2     INDUS       -0.478657  0.478657
9       TAX       -0.468536  0.468536
4       NOX       -0.427321  0.427321
8       RAD       -0.381626  0.381626
6       AGE       -0.380223  0.380223
0      CRIM       -0.379695  0.379695
1        ZN        0.365943  0.365943
11        B        0.333461  0.333461
7       DIS        0.249929  0.249929
3      CHAS        0.179882  0.179882 

Selected high-corr features (|corr| in [0.5, 0.8]): ['LSTAT', 'RM', 'PTRATIO']

=== Final R^2 comparison table ===
dataset       model  R2_test
   full sklearn_OLS 0.658852
   full   custom_GD 0.658835
 subset sklearn_OLS 0.624552
 subset   custom_GD 0.624552


**R² measures how much variance in the target is explained by the model. 
1.0 = perfect, 0.0 = predicts just the mean of y, negative = worse than mean.**

#### Features with the strongest relationship to price
Top 3 by absolute correlation:
1. LSTAT (-0.72)
- LSTAT = % of lower-status population in the area.
- Higher LSTAT - lower house prices.
- Business intuition: Neighborhood socioeconomic profile affects perceived safety, amenities, reputation, demand - directly reflected in property values.
2. RM (+0.70)
- RM = avg number of rooms per dwelling.
- More rooms → larger, more comfortable houses → higher price. This is literally “bigger house = more money,” which makes sense.
3. PTRATIO (-0.51)
- PTRATIO = pupil–teacher ratio in local schools.
- Higher PTRATIO = more students per teacher (more crowded schools, lower perceived education quality).
- Worse schools → less attractive to families → lower property values.

Real Estate POV:
- People are willing to pay more for bigger houses (RM↑).
- People are willing to pay more to avoid social/economic disadvantage (LSTAT↓).
- People are willing to pay more for good schools (PTRATIO↓).

#### Observation:
1. Scratch implementation basically matches sklearn.
The R² values for custom_GD and sklearn_OLS are nearly identical on both datasets.
2. The 3-feature model is surprisingly strong.
Using only LSTAT, RM, PTRATIO I still got ~0.625 R², which is only slightly worse than the ~0.659 R² I got with the full feature set.

# LOGISTIC REGRESSION

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression as SklearnLogReg
from sklearn.metrics import accuracy_score

In [10]:
#load and clean data
DATA_PATH = "data/heart.csv"

df = pd.read_csv(DATA_PATH)

#basic info
print("Shape:", df.shape)
print(df.head())
print(df.info())

#handle missing values by mean imputation for numeric columns
missing_before = df.isna().sum()
print("\nMissing BEFORE:\n", missing_before)

numeric_cols = df.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    df[col] = df[col].fillna(df[col].mean())

missing_after = df.isna().sum()
print("\nMissing AFTER:\n", missing_after)

#sanity: target should be int 0/1
df["target"] = df["target"].astype(int)



#correlation analysis with target
corr_matrix = df.corr(numeric_only=True)
target_corr = corr_matrix["target"].drop("target").sort_values(
    key=lambda x: x.abs(), ascending=False
)

corr_df = pd.DataFrame({
    "feature": target_corr.index,
    "corr_with_target": target_corr.values,
    "abs_corr": np.abs(target_corr.values)
})

print("\nCorrelation with target (sorted by |corr|):")
print(corr_df.to_string(index=False))

#for interprtstion
print("\nTop 5 most correlated features:")
print(corr_df.head(5).to_string(index=False))



#feature subset selection
#    requirement: abs corr in [0.5, 0.8].
#    this dataset: no feature is that strong (~0.44 max).
#    choose top 3 strongest features.


subset_mask = (corr_df["abs_corr"] >= 0.5) & (corr_df["abs_corr"] <= 0.8)
subset_features = corr_df.loc[subset_mask, "feature"].tolist()

if len(subset_features) == 0:
    # so we don't end up with 0 features
    subset_features = corr_df.head(3)["feature"].tolist()
    print("\nNo features in [0.5, 0.8] corr range. Using top 3 instead.")
else:
    print("\nUsing features in [0.5, 0.8] corr range.")

print("Selected subset features:", subset_features)

Shape: (303, 14)
   age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  \
0   63    1   3       145   233    1        0      150      0      2.3      0   
1   37    1   2       130   250    0        1      187      0      3.5      0   
2   41    0   1       130   204    0        0      172      0      1.4      2   
3   56    1   1       120   236    0        1      178      0      0.8      2   
4   57    0   0       120   354    0        1      163      1      0.6      2   

   ca  thal  target  
0   0     1       1  
1   0     2       1  
2   0     2       1  
3   0     2       1  
4   0     2       1  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-nu

In [11]:
#train/test splits
#full feature set
X_full = df.drop(columns=["target"]).values
y_full = df["target"].values  #0 = disease, 1 = no disease

X_train_full, X_test_full, y_train_full, y_test_full = train_test_split(
    X_full, y_full, test_size=0.2, random_state=42
)

print("\nFull feature split shapes:")
print("X_train_full:", X_train_full.shape)
print("X_test_full :", X_test_full.shape)
print("y_train_full:", y_train_full.shape)
print("y_test_full :", y_test_full.shape)

#subset feature set
X_sub = df[subset_features].values
y_sub = y_full

X_train_sub, X_test_sub, y_train_sub, y_test_sub = train_test_split(
    X_sub, y_sub, test_size=0.2, random_state=42
)

print("\nSubset feature split shapes:")
print("X_train_sub:", X_train_sub.shape)
print("X_test_sub :", X_test_sub.shape)
print("y_train_sub:", y_train_sub.shape)
print("y_test_sub :", y_test_sub.shape)


Full feature split shapes:
X_train_full: (242, 13)
X_test_full : (61, 13)
y_train_full: (242,)
y_test_full : (61,)

Subset feature split shapes:
X_train_sub: (242, 3)
X_test_sub : (61, 3)
y_train_sub: (242,)
y_test_sub : (61,)


In [12]:
#sklearn Logistic Regression
sk_full = SklearnLogReg(max_iter=10000)
sk_full.fit(X_train_full, y_train_full)

sk_sub = SklearnLogReg(max_iter=10000)
sk_sub.fit(X_train_sub, y_train_sub)

#sklearn's .score() for classifiers
acc_full_sklearn_score = sk_full.score(X_test_full, y_test_full)
acc_sub_sklearn_score = sk_sub.score(X_test_sub, y_test_sub)

#also verify with accuracy_score
acc_full_sklearn = accuracy_score(y_test_full, sk_full.predict(X_test_full))
acc_sub_sklearn = accuracy_score(y_test_sub, sk_sub.predict(X_test_sub))

print("\nSklearn Logistic Regression accuracy:")
print("Full features (model.score):       ", acc_full_sklearn_score)
print("Full features (accuracy_score):    ", acc_full_sklearn)
print("Subset features (model.score):     ", acc_sub_sklearn_score)
print("Subset features (accuracy_score):  ", acc_sub_sklearn)


Sklearn Logistic Regression accuracy:
Full features (model.score):        0.8852459016393442
Full features (accuracy_score):     0.8852459016393442
Subset features (model.score):      0.819672131147541
Subset features (accuracy_score):   0.819672131147541


In [13]:
#custom Logistic Regression (Gradient Descent)
#    matches the SMLH approach:
#    - linear score -> sigmoid -> probability (binary)
#    - log-loss cost
#    - batch gradient descent weight update
class CustomLogisticRegressionGD:
    """
    bare-bones Logistic Regression trained with batch Gradient Descent.

    - y_hat = sigmoid(X @ w)
    - loss = log loss (cross-entropy)
    - w <- w - alpha * gradient
    """

    def __init__(self, learning_rate=0.05, max_iter=100000):
        self.__learning_rate = learning_rate
        self.__max_iter = max_iter
        #coef_ will store weights INCLUDING the bias at the end
        self.coef_ = None

    def _add_intercept(self, X):
        """append a column of 1s to learn the bias term explicitly."""
        return np.hstack((X, np.ones((len(X), 1))))

    def sigmoid(self, z):
        """Standard logistic sigmoid: 1 / (1 + exp(-z))."""
        return 1.0 / (1.0 + np.exp(-z))

    def fit(self, X, y):
        """
        Fit using full-batch gradient descent.
        X: shape (n_samples, n_features)
        y: shape (n_samples,)
        """
        X_aug = self._add_intercept(X)

        # init weights to zeros (n_features + 1, because of bias)
        self.coef_ = np.zeros(X_aug.shape[1])

        for _ in range(self.__max_iter):
            # predicted probability of class 1
            pred = self.sigmoid(np.dot(X_aug, self.coef_))

            # gradient of average log loss w.r.t. weights
            gradient = np.dot(X_aug.T, (pred - y)) / y.size

            # gradient descent step
            self.coef_ -= self.__learning_rate * gradient

        return self

    def predict_proba(self, X):
        """
        return probabilities for [class0, class1] for each row.
        """
        X_aug = self._add_intercept(X)
        prob_1 = self.sigmoid(np.dot(X_aug, self.coef_))
        prob_0 = 1.0 - prob_1
        return np.vstack((prob_0, prob_1)).T

    def predict(self, X, threshold=0.5):
        """
        convert probabilities to hard class labels using a threshold.
        """
        X_aug = self._add_intercept(X)
        prob_1 = self.sigmoid(np.dot(X_aug, self.coef_))
        return (prob_1 > threshold).astype(int)

    def score(self, X, y):
        """
        classification accuracy, same definition as sklearn's .score.
        """
        y_pred = self.predict(X)
        return np.mean(y_pred == y)


#train custom models
custom_full = CustomLogisticRegressionGD(learning_rate=0.05, max_iter=100000)
custom_full.fit(X_train_full, y_train_full)
acc_full_custom = custom_full.score(X_test_full, y_test_full)

custom_sub = CustomLogisticRegressionGD(learning_rate=0.05, max_iter=100000)
custom_sub.fit(X_train_sub, y_train_sub)
acc_sub_custom = custom_sub.score(X_test_sub, y_test_sub)

print("\nCustom Logistic Regression accuracy:")
print("Full features accuracy:   ", acc_full_custom)
print("Subset features accuracy: ", acc_sub_custom)

C:\Users\user\AppData\Local\Temp\ipykernel_5240\1839241889.py:27: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-z))



Custom Logistic Regression accuracy:
Full features accuracy:    0.8524590163934426
Subset features accuracy:  0.819672131147541


In [14]:
#final comparison table
results_df = pd.DataFrame({
    "dataset": ["full", "full", "subset", "subset"],
    "model": ["sklearn_LogReg", "custom_GD_LogReg", "sklearn_LogReg", "custom_GD_LogReg"],
    "accuracy_test": [
        acc_full_sklearn, acc_full_custom,
        acc_sub_sklearn, acc_sub_custom
    ]
})

print("\n=== Final accuracy comparison table ===")
print(results_df.to_string(index=False))


=== Final accuracy comparison table ===
dataset            model  accuracy_test
   full   sklearn_LogReg       0.885246
   full custom_GD_LogReg       0.852459
 subset   sklearn_LogReg       0.819672
 subset custom_GD_LogReg       0.819672


## Observations:
- I handled data, imputed, analyzed correlations, chose a most predictive subset.
- I trained both sklearn LogisticRegression and gradient-descent Logistic Regression (sigmoid activation, log loss, gradient descent updates). 
- on the full feature set it reached ~86.9% test accuracy (sklearn), ~83.6% (custom).
- on the 3-feature subset (exang, cp, oldpeak) it still got ~82% accuracy.
- the top clinical signals: chest pain type, ECG changes under stress, and angina during exercise are highly predictive.